In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np

from sklearn.linear_model import LogisticRegression

from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score,confusion_matrix, ConfusionMatrixDisplay, classification_report

In [ ]:
def LReg(df):
    X = df.drop('diagnostico', axis=1)
    y = df['diagnostico']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

    class_weight = {0: sum(y == 1) / len(y), 1: sum(y == 0) / len(y)}

    params_grid = {'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
    'C' : np.logspace(-4, 4, 20),
    'solver' : ['lbfgs','newton-cg','liblinear','sag','saga'],
    'max_iter' : [100, 1000,2500, 5000]
    }

    classifier = LogisticRegression(class_weight=class_weight, random_state=42)

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    grid_classifier = GridSearchCV(classifier, params_grid, scoring='accuracy', cv=cv)
    grid_classifier.fit(X_val, y_val)
    
    best_parameters = grid_classifier.best_params_

    model = LogisticRegression(**best_parameters, class_weight=class_weight, random_state=42)

    model.fit(X_train, y_train)
    
    pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, pred)
    
    print(test_accuracy)    
    print(classification_report(y_test,pred))
    print(confusion_matrix(y_test, pred))

    plt.figure(figsize=(8, 6))
    sns.heatmap(confusion_matrix(y_test, pred), annot=True, fmt='d', cmap='Blues', xticklabels=['Sem Sepse', 'Com Sepse'], yticklabels=['Sem Sepse', 'Com Sepse'])
    plt.xlabel('Predito')
    plt.ylabel('Verdadeiro')
    plt.title('Matriz de Confusão')
    plt.show()


In [ ]:
def LReg_Over(df):
    X = df.drop('diagnostico', axis=1)
    y = df['diagnostico']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

    ros = RandomOverSampler(random_state=42)
    X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

    class_weight = {0: sum(y == 1) / len(y), 1: sum(y == 0) / len(y)}

    params_grid = {'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
    'C' : np.logspace(-4, 4, 20),
    'solver' : ['lbfgs','newton-cg','liblinear','sag','saga'],
    'max_iter' : [100, 1000,2500, 5000]
    }

    classifier = LogisticRegression(class_weight=class_weight, random_state=42)

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    grid_classifier = GridSearchCV(classifier, params_grid, scoring='accuracy', cv=cv)
    grid_classifier.fit(X_val, y_val)
    
    best_parameters = grid_classifier.best_params_

    model = LogisticRegression(**best_parameters, class_weight=class_weight, random_state=42)

    model.fit(X_train_resampled, y_train_resampled)
    
    pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, pred)
    
    print(test_accuracy)    
    print(classification_report(y_test,pred))
    print(confusion_matrix(y_test, pred))

    plt.figure(figsize=(8, 6))
    sns.heatmap(confusion_matrix(y_test, pred), annot=True, fmt='d', cmap='Blues', xticklabels=['Sem Sepse', 'Com Sepse'], yticklabels=['Sem Sepse', 'Com Sepse'])
    plt.xlabel('Predito')
    plt.ylabel('Verdadeiro')
    plt.title('Matriz de Confusão')
    plt.show()

Valores Normais

In [ ]:
df_normal = pd.read_csv('df_normal.csv')

In [ ]:
LReg(df_normal)

In [ ]:
LReg_Over(df_normal)

KNN

In [ ]:
df_knn = pd.read_csv('df_knn.csv')

In [ ]:
LReg(df_knn)

In [ ]:
LReg_Over(df_knn)

Media

In [ ]:
df_media = pd.read_csv('df_media.csv')

In [ ]:
LReg(df_media)

In [ ]:
LReg_Over(df_media)

Mediana

In [ ]:
df_mediana = pd.read_csv('df_mediana.csv')

In [ ]:
LReg(df_mediana)

In [ ]:
LReg_Over(df_mediana)

Moda

In [ ]:
df_moda = pd.read_csv('df_moda.csv')

In [ ]:
LReg(df_moda)

In [ ]:
LReg_Over(df_moda)

Regressão

In [ ]:
df_regressao = pd.read_csv('df_regressao.csv')

In [ ]:
LReg(df_regressao)

In [ ]:
LReg_Over(df_regressao)